In [ ]:
import os
from pathlib import Path
from PIL import Image


In [ ]:
from source.visual_genome_meta_data import read_json_to_dict
from source.visual_genome_meta_data import get_image_meta_data
from source.visual_genome_meta_data import count_occurrences
from source.visual_genome_to_yolo import create_class_mapping_from_list
from source.visual_genome_to_yolo import save_class_map_to_yaml
from source.visual_genome_to_yolo import convert_single_image_to_yolo
from source.visual_genome_to_yolo import read_yaml_to_class_map
from source.visual_genome_to_yolo import read_yolo_metadata
# from source.visual_genome_to_yolo import visual_genome_to_yolo_data
from source.visual_genome_to_yolo import visual_genome_to_yolo_data_n
from source.visual_genome_meta_data import plot_image_with_multiple_bboxes
from source.visual_genome_meta_data import get_image_ids
from source.yolo_training_structure import distribute_train_val_files as dist_train_val

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random
import os

def simulate_old_photo_effects(image_path, output_path, intensity=1.0):
    """
    Apply various degradation effects to simulate old photos.
    
    Args:
        image_path: Path to input image
        output_path: Path to save processed image
        intensity: Float 0-1, controls how strong the effects are
    """
    # Load image
    img = Image.open(image_path)
    
    # Convert to RGB if needed
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    # 1. Reduce image quality (JPEG compression artifacts)
    if random.random() < 0.7:  # Apply 70% of the time
        quality = int(30 + (70 * (1 - intensity)))  # Quality between 30-100
        img.save('temp.jpg', 'JPEG', quality=quality)
        img = Image.open('temp.jpg')
        os.remove('temp.jpg')
    
    # 2. Add noise
    if random.random() < 0.6:
        img_array = np.array(img)
        noise = np.random.normal(0, 10 * intensity, img_array.shape).astype(np.uint8)
        img_array = np.clip(img_array.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        img = Image.fromarray(img_array)
    
    # 3. Blur (camera shake, focus issues)
    if random.random() < 0.5:
        blur_radius = 0.5 + (2 * intensity)
        img = img.filter(ImageFilter.GaussianBlur(radius=blur_radius))
    
    # 4. Adjust brightness (faded photos)
    if random.random() < 0.8:
        brightness_factor = 0.7 + (0.4 * random.random()) - (0.2 * intensity)
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(brightness_factor)
    
    # 5. Adjust contrast (faded photos)
    if random.random() < 0.8:
        contrast_factor = 0.6 + (0.6 * random.random()) - (0.3 * intensity)
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(contrast_factor)
    
    # 6. Color shift (aging effects)
    if random.random() < 0.6:
        img_array = np.array(img).astype(np.float32)
        # Add yellowish tint
        img_array[:, :, 0] *= (1.0 + 0.1 * intensity)  # Red
        img_array[:, :, 1] *= (1.0 + 0.05 * intensity)  # Green
        img_array[:, :, 2] *= (1.0 - 0.1 * intensity)  # Blue (reduce)
        img_array = np.clip(img_array, 0, 255).astype(np.uint8)
        img = Image.fromarray(img_array)
    
    # 7. Add scratches/spots (random lines)
    if random.random() < 0.3:
        img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        for _ in range(random.randint(1, 5)):
            pt1 = (random.randint(0, img.width), random.randint(0, img.height))
            pt2 = (random.randint(0, img.width), random.randint(0, img.height))
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            cv2.line(img_cv, pt1, pt2, color, random.randint(1, 3))
        img = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
    
    # 8. Vignetting (darker edges)
    if random.random() < 0.4:
        img_array = np.array(img).astype(np.float32)
        h, w = img_array.shape[:2]
        
        # Create vignette mask
        X, Y = np.meshgrid(np.linspace(-1, 1, w), np.linspace(-1, 1, h))
        radius = np.sqrt(X**2 + Y**2)
        vignette = 1 - (radius * intensity * 0.3)
        vignette = np.clip(vignette, 0.5, 1.0)
        
        # Apply vignette
        for i in range(3):
            img_array[:, :, i] *= vignette
        
        img_array = np.clip(img_array, 0, 255).astype(np.uint8)
        img = Image.fromarray(img_array)
    
    # Save the processed image
    img.save(output_path, 'JPEG', quality=95)
    return img


def simulate_specific_old_effects(image_path, output_path):
    """
    Apply specific effects that match your old photos.
    Adjust these based on what you observe in your test images.
    """
    img = Image.open(image_path).convert('RGB')
    
    # Heavy JPEG compression (very low quality)
    img.save('temp.jpg', 'JPEG', quality=15)
    img = Image.open('temp.jpg')
    os.remove('temp.jpg')
    
    # Significant brightness reduction
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(0.8)
    
    # Low contrast
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(0.9)
    
    # Add significant noise
    img_array = np.array(img)
    noise = np.random.normal(0, 0.1, img_array.shape).astype(np.uint8)
    img_array = np.clip(img_array.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    img = Image.fromarray(img_array)
    
    # Strong blur
    img = img.filter(ImageFilter.GaussianBlur(radius=1.5))
    
    img.save(output_path, 'JPEG', quality=85) 





In [ ]:
np.random.normal(0, 1, 3)

In [ ]:
def process_training_dataset(input_dir, output_dir, augmentation_ratio=0.5):
    """
    Process a directory of training images to simulate old photo effects.
    
    Args:
        input_dir: Directory with original images
        output_dir: Directory to save processed images
        augmentation_ratio: Fraction of images to augment (0.5 = 50%)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif'))]
    
    for img_file in image_files:
        input_path = os.path.join(input_dir, img_file)
        
        # Always copy original
        original_output = os.path.join(output_dir, img_file)
        #img = Image.open(input_path)
        #img.save(original_output)
        
        # Maybe add augmented version
        if random.random() < augmentation_ratio:
            # Create filename for augmented version
            name, ext = os.path.splitext(img_file)
            aug_filename = f"{name}_aged{ext}"
            aug_output = os.path.join(output_dir, aug_filename)
            #aug_output = os.path.join(output_dir, img_file)
            
            # Apply aging effects with random intensity
            intensity = random.uniform(0.3, 0.8)
            simulate_old_photo_effects(input_path, aug_output, intensity)
    
    print(f"Processed {len(image_files)} images in {input_dir}")

In [ ]:
def process_training_dataset_spec(input_dir, output_dir, augmentation_ratio=0.5):
    """
    Process a directory of training images to simulate old photo effects.
    
    Args:
        input_dir: Directory with original images
        output_dir: Directory to save processed images
        augmentation_ratio: Fraction of images to augment (0.5 = 50%)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif'))]
    
    for img_file in image_files:
        input_path = os.path.join(input_dir, img_file)
        
        # Always copy original
        original_output = os.path.join(output_dir, img_file)
        #img = Image.open(input_path)
        #img.save(original_output)
        

        # Create filename for augmented version
        name, ext = os.path.splitext(img_file)
        aug_filename = f"{name}_aged{ext}"
        aug_output = os.path.join(output_dir, aug_filename)
        #aug_output = os.path.join(output_dir, img_file)
        
        # Apply aging effects with random intensity
        intensity = random.uniform(0.3, 0.8)
        simulate_specific_old_effects(input_path, aug_output)
    
    print(f"Processed {len(image_files)} images in {input_dir}")

In [ ]:
import os
from pathlib import Path

# Set root path (choose one):
# Option A: Use current working directory
root_path = Path.cwd()
root_path

In [ ]:
data_path = root_path /'..'/'test_old_photo_effect'
originals_path = data_path / 'originals'

spec_old_effect_path = data_path / 'old_effect_spec'


In [ ]:
#print(originals_train_path)
#print(oldened_train_path)

In [ ]:
#process_training_dataset(originals_path, old_effect_path, augmentation_ratio=0.3)
process_training_dataset_spec(originals_path, spec_old_effect_path)

In [ ]:
'_'.join(['skede', 'dke'])

In [ ]:
'_'.join(['skede'])